# Packages

In [7]:
import cv2                                # State-of-the-art computer vision algorithms library
import math                               # Fundamental package for mathematics application
import numpy as np                        # Fundamental package for scientific computing
import mediapipe as mp                    # Pose estimation through mp
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
from ultralytics import YOLO              # Manage the YOLO models

# Setup YOLO

In [4]:
# Define the class for YOLO 
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

model_yolo = YOLO("yolo-Weights/yolov8n.pt")

# Stream Feed

## Stream Setup

In [5]:
# Initialize to retrieve the camera flow
pipe = rs.pipeline()
cfg = rs.config()

# Define the format of both stream
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Align both cameras
align = rs.align(rs.stream.color)

## Stream Test

In [6]:
# Start the capture
pipe.start(cfg)

while True:
  # Wait for a coherent pair of frames "depth and color" and align them
  frame = pipe.wait_for_frames()
  aligned_frame = align.process(frame)
  
  # Retrieve the depth and color flow
  depth_frame = frame.get_depth_frame()
  color_frame = frame.get_color_frame()
  
  # Apply filters to reduce the treatment charge by decreasing the resolution
  decimation = rs.decimation_filter()
  decimation.set_option(rs.option.filter_magnitude, 1)
  
  # Apply smothering filter
  spatial = rs.spatial_filter()
  spatial.set_option(rs.option.filter_magnitude, 5)
  spatial.set_option(rs.option.filter_smooth_alpha, 1)
  spatial.set_option(rs.option.filter_smooth_delta, 50)
  
  # Apply filters to fill the holes
  hole_filling = rs.hole_filling_filter()
  
  # Retrieve the images from both flow (colorized depth)
  colorizer = rs.colorizer()
  color_image = np.asanyarray(color_frame.get_data())
  decimated_depth = decimation.process(depth_frame)
  smoothed_depth = spatial.process(decimated_depth)
  filled_depth = hole_filling.process(smoothed_depth)
  colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())
  
  cv2.imshow("Depth", colorized_depth_frame)
  cv2.imshow("Color", color_image)
  
  if cv2.waitKey(1) == ord('q'):
    break
      
pipe.stop()
cv2.destroyAllWindows()

# MediaPipe Application

In [15]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [18]:
# Setup mediapipe instance
with mp_pose.Pose() as pose:
    # Start the capture
    pipe.start(cfg)
    
    while True:
        # Wait for a coherent pair of frames "depth and color" and align them
        frame = pipe.wait_for_frames()
        aligned_frame = align.process(frame)
        
        # Retrieve the depth and color flow
        depth_frame = frame.get_depth_frame()
        color_frame = frame.get_color_frame()
        
        # Apply filters to reduce the treatment charge by decreasing the resolution
        decimation = rs.decimation_filter()
        decimation.set_option(rs.option.filter_magnitude, 1)
        
        # Apply smothering filter
        spatial = rs.spatial_filter()
        spatial.set_option(rs.option.filter_magnitude, 5)
        spatial.set_option(rs.option.filter_smooth_alpha, 1)
        spatial.set_option(rs.option.filter_smooth_delta, 50)
        
        # Apply filters to fill the holes
        hole_filling = rs.hole_filling_filter()
        
        # Retrieve the images from both flow (colorized depth)
        colorizer = rs.colorizer()
        color_image = np.asanyarray(color_frame.get_data())
        
        decimated_depth = decimation.process(depth_frame)
        smoothed_depth = spatial.process(decimated_depth)
        filled_depth = hole_filling.process(smoothed_depth)
        depth_image = np.asanyarray(filled_depth.get_data())
        colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())
        
        # Convert color image to RGB
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        
        # Add MP on the colored image
        results = pose.process(color_image_rgb)
        
        # Render detections
        mp_drawing.draw_landmarks(color_image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )
        
        # Convert RGB image back to BGR for displaying
        color_image_bgr = cv2.cvtColor(color_image_rgb, cv2.COLOR_RGB2BGR)
        
        # Retrieve and print the coordinates for each landmark
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                x = int(landmark.x * color_image.shape[1]) 
                y = int(landmark.y * color_image.shape[0])  
                
                # Print the calculated coordinates
                print(f"X = {x}, Y = {y}")
                
                # Ensure that the calculated pixel coordinates are within the bounds of the depth frame
                if 0 <= y < color_image.shape[0] and 0 <= x < color_image.shape[1]:
                    depth_value = depth_image[y, x]
                    print(f"Z = {depth_value}")
                else:
                    print("Coordinates out of bounds")

        cv2.imshow("Depth", colorized_depth_frame)
        cv2.imshow("Color", color_image)
        cv2.imshow("MediaPipe", color_image_bgr)
        
        if cv2.waitKey(1) == ord('q'):
            break
            
    pipe.stop()
    cv2.destroyAllWindows()

X = 338, Y = 105
Z = 611
X = 342, Y = 93
Z = 610
X = 345, Y = 93
Z = 609
X = 348, Y = 94
Z = 610
X = 329, Y = 93
Z = 606
X = 323, Y = 92
Z = 597
X = 316, Y = 92
Z = 588
X = 347, Y = 98
Z = 610
X = 302, Y = 93
Z = 583
X = 341, Y = 120
Z = 614
X = 326, Y = 119
Z = 621
X = 347, Y = 138
Z = 625
X = 280, Y = 118
Z = 4470
X = 349, Y = 168
Z = 659
X = 258, Y = 138
Z = 4470
X = 338, Y = 176
Z = 681
X = 268, Y = 149
Z = 686
X = 333, Y = 185
Z = 809
X = 267, Y = 154
Z = 674
X = 333, Y = 177
Z = 681
X = 271, Y = 151
Z = 674
X = 330, Y = 172
Z = 679
X = 273, Y = 149
Z = 674
X = 303, Y = 196
Z = 490
X = 255, Y = 185
Z = 547
X = 340, Y = 224
Z = 744
X = 245, Y = 195
Z = 520
X = 350, Y = 272
Z = 748
X = 258, Y = 227
Z = 518
X = 342, Y = 281
Z = 704
X = 261, Y = 233
Z = 432
X = 369, Y = 295
Z = 845
X = 257, Y = 246
Z = 428
X = 353, Y = 127
Z = 618
X = 354, Y = 112
Z = 613
X = 357, Y = 114
Z = 614
X = 359, Y = 116
Z = 617
X = 342, Y = 106
Z = 610
X = 333, Y = 103
Z = 607
X = 323, Y = 101
Z = 586
X = 34

# YOLO Application

In [32]:
# Setup mediapipe instance
with mp_pose.Pose() as pose:
    # Start the capture
    pipe.start(cfg)
    
    while True:
        # Wait for a coherent pair of frames "depth and color" and align them
        frame = pipe.wait_for_frames()
        aligned_frame = align.process(frame)
        
        # Retrieve the depth and color flow
        depth_frame = frame.get_depth_frame()
        color_frame = frame.get_color_frame()
        
        # Apply filters to reduce the treatment charge by decreasing the resolution
        decimation = rs.decimation_filter()
        decimation.set_option(rs.option.filter_magnitude, 1)
        
        # Apply smothering filter
        spatial = rs.spatial_filter()
        spatial.set_option(rs.option.filter_magnitude, 5)
        spatial.set_option(rs.option.filter_smooth_alpha, 1)
        spatial.set_option(rs.option.filter_smooth_delta, 50)
        
        # Apply filters to fill the holes
        hole_filling = rs.hole_filling_filter()
        
        # Retrieve the images from both flow (colorized depth)
        colorizer = rs.colorizer()
        color_image = np.asanyarray(color_frame.get_data())
        decimated_depth = decimation.process(depth_frame)
        smoothed_depth = spatial.process(decimated_depth)
        filled_depth = hole_filling.process(smoothed_depth)
        depth_image = np.asanyarray(filled_depth.get_data())
        colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())
        
        # Convert color image to RGB
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image_rgb.flags.writeable = False
        
        # Add MP on the colored image
        results = pose.process(color_image_rgb)
        
        # Convert RGB image back to BGR for displaying
        color_image_rgb.flags.writeable = True
        color_image_bgr = cv2.cvtColor(color_image_rgb, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(color_image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )
        
        # Retrieve and print the coordinates for each landmark
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                x = int(landmark.x * color_image.shape[1]) 
                y = int(landmark.y * color_image.shape[0])  
                
                # Print the calculated coordinates
                print(f"X = {x}, Y = {y}")
                
                # Ensure that the calculated pixel coordinates are within the bounds of the depth frame
                if 0 <= y < color_image.shape[0] and 0 <= x < color_image.shape[1]:
                    depth_value = depth_image[y, x]
                    print(f"Z = {depth_value}")
                else:
                    print("Coordinates out of bounds")
                    
        # Apply YOLOv8
        results_yolo = model_yolo(color_image_rgb, stream=True)
        
        # Display YOLO detections
        for r in results_yolo:
            boxes = r.boxes
            for box in boxes:
                # Bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # Draw bounding box
                cv2.rectangle(color_image_rgb, (x1, y1), (x2, y2), (255, 0, 255), 3)
                # Confidence and class
                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])
                class_name = classNames[cls]
                # Display class and confidence
                org = (x1, y1)
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                color = (255, 0, 0)
                thickness = 2
                cv2.putText(color_image_rgb, f"{class_name}: {confidence}", org, font, fontScale, color, thickness)

        cv2.imshow("Depth", colorized_depth_frame)
        cv2.imshow("Color", color_image)
        cv2.imshow("MediaPipe&YOLO", color_image_rgb)
        
        if cv2.waitKey(1) == ord('q'):
            break
            
    pipe.stop()
    cv2.destroyAllWindows()

0: 480x640 2 persons, 1 tie, 1 bottle, 1 keyboard, 115.6ms
Speed: 3.1ms preprocess, 115.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 1 umbrella, 1 laptop, 134.1ms
Speed: 2.0ms preprocess, 134.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 1 umbrella, 1 laptop, 109.6ms
Speed: 1.1ms preprocess, 109.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 1 cup, 1 laptop, 155.5ms
Speed: 6.0ms preprocess, 155.5ms inference, 5.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 129.0ms
Speed: 2.6ms preprocess, 129.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 2 laptops, 183.3ms
Speed: 2.0ms preprocess, 183.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
X = 351, Y = 113
Z = 631
X = 347, Y = 94
Z = 627
X = 349, Y = 94
Z = 628
X = 350, Y = 93
Z = 629
X = 336, Y = 94
Z = 623
X = 330, Y = 9